In [2]:
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
from torch.quantization import quantize_fx
import numpy as np
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")


def save_architecture_txt(model,dir_path,filename):
    complete_file_name = os.path.join(dir_path, filename+"_arch.txt")
    with open(complete_file_name, "w") as f:
            f.write(str(model))
            f.close()
    return None



x, y = load_iris(return_X_y=True)
train_X, test_X, train_y, test_y = train_test_split(x, y, test_size=0.2)
train_X = Variable(torch.Tensor(train_X).float())
test_X = Variable(torch.Tensor(test_X).float())
train_y = Variable(torch.Tensor(train_y).long())
test_y = Variable(torch.Tensor(test_y).long())


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1) 

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        #X = self.softmax(X)
        return X

model = Net()
model.train()

n_epochs = 100
opt = torch.optim.SGD(model.parameters(), lr=0.1)
loss_fn = torch.nn.CrossEntropyLoss()


for epoch in range(n_epochs):
    opt.zero_grad()
    out = model(train_X)
    loss = loss_fn(out, train_y)
    loss.backward()
    opt.step()


torch.save(model, "../../results/iris.pth")
dir_path = '../../results'
filename = 'linear_nn'
save_architecture_txt(model, dir_path, filename)


dataiter = iter(train_X)
data = next(dataiter)
m = copy.deepcopy(model)
m.eval()
#qconfig_dict = {"": torch.quantization.get_default_qconfig("fbgemm")}
qconfig_dict = torch.ao.quantization.qconfig_mapping.get_default_qconfig_mapping("fbgemm") 
model_prepared = quantize_fx.prepare_fx(m, qconfig_dict, data)

with torch.inference_mode():
    for _ in range(10):
        data = next(dataiter)
        model_prepared(data)
model_quantized = quantize_fx.convert_fx(model_prepared)
torch.save(model_quantized, "../../results/PTQ_static_iris.pth")
dir_path = '../../results'
filename = 'linear_nn_quant'
save_architecture_txt(model_quantized, dir_path, filename)

In [3]:
std_model = torch.load("../../results/iris.pth")
std_model.eval()


true_labels = []
predicted_labels = []
softmax_confidences = []


with torch.no_grad():
    logits = std_model(test_X)
    outputs = torch.softmax(logits,dim=1)
    confidences,predictions = torch.max(outputs,1)
    softmax_confidences.extend(confidences.cpu().numpy())
    predicted_labels.extend(predictions.cpu().numpy())
    true_labels.extend(test_y.cpu().numpy())
accuracy = round(accuracy_score(np.array(true_labels),np.array(predicted_labels)),3)
    

print("True label")
print(true_labels)
print("--------------------------------------------")

print("Inferencing - Standard Model")
print(predicted_labels)
print(accuracy)
print("--------------------------------------------")







quant_model = torch.load("../../results/PTQ_static_iris.pth")
quant_model.eval()


true_labels = []
predicted_labels = []
softmax_confidences = []


with torch.no_grad():
    logits = quant_model(test_X)
    outputs = torch.softmax(logits,dim=1)
    confidences,predictions = torch.max(outputs,1)
    softmax_confidences.extend(confidences.cpu().numpy())
    predicted_labels.extend(predictions.cpu().numpy())
    true_labels.extend(test_y.cpu().numpy())
accuracy = round(accuracy_score(np.array(true_labels),np.array(predicted_labels)),3)

print("Inferencing - Quantised Model")
print(predicted_labels)
print(accuracy)

True label
[1, 2, 1, 0, 1, 1, 2, 2, 0, 0, 2, 2, 1, 1, 0, 2, 1, 1, 1, 0, 2, 1, 1, 2, 1, 2, 0, 0, 1, 2]
--------------------------------------------
Inferencing - Standard Model
[2, 2, 2, 0, 1, 2, 2, 2, 0, 0, 2, 2, 1, 1, 0, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2]
0.7
--------------------------------------------
Inferencing - Quantised Model
[2, 2, 2, 0, 1, 2, 2, 2, 0, 0, 2, 2, 1, 1, 0, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2]
0.7


In [4]:
import torch
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
from torch.quantization import QuantStub, DeQuantStub

x, y = load_iris(return_X_y=True)
train_X, test_X, train_y, test_y = train_test_split(x, y, test_size=0.8)
train_X = Variable(torch.Tensor(train_X).float())
test_X = Variable(torch.Tensor(test_X).float())
train_y = Variable(torch.Tensor(train_y).long())
test_y = Variable(torch.Tensor(test_y).long())


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1)
        self.quant = QuantStub() 
        self.dequant = DeQuantStub() 

    def forward(self, X):
        X = self.quant(X)
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)
        X = self.dequant(X)
        return X

m = Net()

m.train()
backend = "fbgemm"
m.qconfig = torch.quantization.get_default_qconfig(backend)
torch.quantization.prepare_qat(m, inplace=True)

n_epochs = 10
opt = torch.optim.SGD(m.parameters(), lr=0.1)
loss_fn = torch.nn.CrossEntropyLoss()


for epoch in range(n_epochs):
    opt.zero_grad()
    out = m(train_X)
    loss = loss_fn(out, train_y)
    loss.backward()
    opt.step()
    
m.eval()
model_quantized = torch.quantization.convert(m, inplace=True)
torch.save(model_quantized, '../../results/QAT_iris.pth')